In [1]:
import requests
url = "https://www.naukri.com/data-science-jobs"
resp=requests.get(url)

C:\Users\Shara\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


NameError: name 'url' is not defined

In [170]:
from bs4 import BeautifulSoup
import pandas as pd

In [171]:
import os
os.chdir("../chrome_driver")
from selenium import webdriver
#browser2 = webdriver.Chrome()

In [156]:
browser2.get(url)

In [157]:
raw_html=browser2.page_source

In [158]:
parsed_html=BeautifulSoup(raw_html,"html.parser")

In [175]:
Naukri_dsjobs = pd.DataFrame(columns=("Job_id","Company","Experience","Salary","Location","Job_link","Skillset","Posted"))

In [187]:
def get_data(parsed_html):
    table = pd.DataFrame()
    job_id = []
    company_name=[]
    Exp = []
    Salary = []
    Location = []
    Link = []
    Skillset = []
    Posted = []
    for row in parsed_html.find_all("article",class_="jobTuple bgWhite br4 mb-8"):
        job_id.append(row["data-job-id"])
        company_name.append(row.find("div",class_="mt-7 companyInfo subheading lh16").a.get_text())
        Link.append(row.find("div",class_="mt-7 companyInfo subheading lh16").a['href'])
        try:
            Exp.append(row.find("li",class_="fleft grey-text br2 placeHolderLi experience").get_text())
        except:
            Exp.append("No Data")
        try:
            Salary.append(row.find("li",class_="fleft grey-text br2 placeHolderLi salary").get_text())
        except:
            Salary.append("No data")
        try:
            Location.append(row.find("li",class_="fleft grey-text br2 placeHolderLi location").get_text())
        except:
            Location.append("No Data")
        try:
            skills = row.find_all("li",class_="fleft fs12 grey-text lh16 dot")
            Skillset.append((list(map(lambda x: x.text,skills))))
        except:
            Skillset.append("No Data")
        try:
            Posted.append(row.find("div",class_="jobTupleFooter mt-20").span.get_text())
        except:
            Posted.append("No Data")

        table = pd.DataFrame({"Job_id":job_id,"Company":company_name,"Experience":Exp,"Salary":Salary,"Location":Location,"Job_link":Link,"Skillset":Skillset,"Posted":Posted})
    return table

In [188]:
import time
from tqdm import tqdm
base_url = "https://www.naukri.com/data-science-jobs-{}"
for page_num in tqdm(range(401,1001)):
    url=base_url.format(page_num)
    browser1 = webdriver.Chrome()
    browser1.get(url)
    time.sleep(1)
    raw_data=browser1.page_source
    browser1.close()
    html=BeautifulSoup(raw_data,"html.parser")
    Naukri_dsjobs = Naukri_dsjobs.append(get_data(html),ignore_index = True)































  0%|                                                                                          | 0/600 [00:00<?, ?it/s]




























  0%|▏                                                                               | 1/600 [00:11<1:57:48, 11.80s/it]




























  0%|▎                                                                               | 2/600 [00:42<2:53:48, 17.44s/it]




























  0%|▍                                                                               | 3/600 [00:54<2:38:35, 15.94s/it]




























  1%|▌                                                                               | 4/600 [01:07<2:28:18, 14.93s/it]




























  1%|▋                                                                               | 5/600 [01:26<2:41:27, 16.28s/it]




























  1%|▊                                                                             

 18%|██████████████▎                                                               | 110/600 [24:42<1:48:43, 13.31s/it]




























 18%|██████████████▍                                                               | 111/600 [25:00<1:58:09, 14.50s/it]




























 19%|██████████████▌                                                               | 112/600 [25:13<1:55:36, 14.22s/it]




























 19%|██████████████▋                                                               | 113/600 [25:34<2:12:29, 16.32s/it]




























 19%|██████████████▊                                                               | 114/600 [25:45<1:58:58, 14.69s/it]




























 19%|██████████████▉                                                               | 115/600 [25:57<1:51:06, 13.74s/it]




























 19%|███████████████                                                               | 116/600 [26:08<1:44:16, 12.

 37%|████████████████████████████▌                                                 | 220/600 [50:10<1:44:26, 16.49s/it]




























 37%|████████████████████████████▋                                                 | 221/600 [50:18<1:28:07, 13.95s/it]




























 37%|████████████████████████████▊                                                 | 222/600 [50:29<1:22:28, 13.09s/it]




























 37%|████████████████████████████▉                                                 | 223/600 [50:41<1:20:19, 12.78s/it]




























 37%|█████████████████████████████                                                 | 224/600 [51:02<1:34:55, 15.15s/it]




























 38%|█████████████████████████████▎                                                | 225/600 [51:14<1:28:43, 14.20s/it]




























 38%|█████████████████████████████▍                                                | 226/600 [51:31<1:33:08, 14.

 55%|█████████████████████████████████████████▊                                  | 330/600 [1:14:40<1:08:47, 15.29s/it]




























 55%|█████████████████████████████████████████▉                                  | 331/600 [1:14:51<1:02:51, 14.02s/it]




























 55%|██████████████████████████████████████████                                  | 332/600 [1:15:03<1:00:09, 13.47s/it]




























 56%|███████████████████████████████████████████▎                                  | 333/600 [1:15:14<56:37, 12.72s/it]




























 56%|███████████████████████████████████████████▍                                  | 334/600 [1:15:26<55:43, 12.57s/it]




























 56%|███████████████████████████████████████████▌                                  | 335/600 [1:15:38<54:47, 12.41s/it]




























 56%|██████████████████████████████████████████▌                                 | 336/600 [1:15:57<1:03:47, 14.

 73%|█████████████████████████████████████████████████████████▏                    | 440/600 [1:39:34<38:17, 14.36s/it]




























 74%|█████████████████████████████████████████████████████████▎                    | 441/600 [1:39:46<36:08, 13.64s/it]




























 74%|█████████████████████████████████████████████████████████▍                    | 442/600 [1:39:56<33:21, 12.67s/it]




























 74%|█████████████████████████████████████████████████████████▌                    | 443/600 [1:40:09<33:00, 12.61s/it]




























 74%|█████████████████████████████████████████████████████████▋                    | 444/600 [1:40:20<32:13, 12.39s/it]




























 74%|█████████████████████████████████████████████████████████▊                    | 445/600 [1:40:32<30:59, 12.00s/it]




























 74%|█████████████████████████████████████████████████████████▉                    | 446/600 [1:40:44<31:28, 12.

 92%|███████████████████████████████████████████████████████████████████████▌      | 550/600 [2:05:53<11:09, 13.40s/it]




























 92%|███████████████████████████████████████████████████████████████████████▋      | 551/600 [2:06:08<11:17, 13.83s/it]




























 92%|███████████████████████████████████████████████████████████████████████▊      | 552/600 [2:06:20<10:39, 13.33s/it]




























 92%|███████████████████████████████████████████████████████████████████████▉      | 553/600 [2:06:36<11:06, 14.18s/it]




























 92%|████████████████████████████████████████████████████████████████████████      | 554/600 [2:06:53<11:22, 14.84s/it]




























 92%|████████████████████████████████████████████████████████████████████████▏     | 555/600 [2:07:04<10:24, 13.89s/it]




























 93%|████████████████████████████████████████████████████████████████████████▎     | 556/600 [2:07:15<09:24, 12.

In [189]:
Naukri_dsjobs.shape

(5735, 8)

In [190]:
Naukri_dsjobs.to_csv('naukri.csv')

In [192]:
os.getcwd()

'E:\\soumya\\herovired\\idma_ft-main\\chrome_driver'